## Ability of models to reproduce regional patterns

In [ ]:
chunk_start

**Note**: These comparisons are only exact when the model grids are identical. Otherwise, the comparisons are based on the overlap between the model grids and the region of interest, and the results may not be totally comparable.

In [ ]:
# model_dict = {
#     "LOCATE":"/data/proteus1/scratch/gle/getmval/locate",
#     "GETM":"/data/proteus1/scratch/rwi/adhoc/getm/3dmn"
# }

model_dict = model_dict_str
num_models = len(model_dict)

In [ ]:
# step 1 is to figure out the variables available
variables = []
for dir_name in list(model_dict.values()):
    variables += [os.path.basename(x).split("_")[1] for x in glob.glob(dir_name + "/results/regionals/*_*_regionals.csv")]

# restrict variables to those that occur twice
variables = list(set([x for x in variables if variables.count(x) == 2]))


In [ ]:

for vv in variables:
  try:
    df1 = pd.read_csv(glob.glob(model_dict[list(model_dict.keys())[0]] + "/results/regionals/nsbc_" + vv + "_regionals.csv")[0])
    key = list(model_dict.keys())[0]
    # change model to key in variable
    df1["variable"] = [x if x != "model" else key for x in df1["variable"]]

    df2 = pd.read_csv(glob.glob(model_dict[list(model_dict.keys())[1]] + "/results/regionals/nsbc_" + vv + "_regionals.csv")[0])
    key = list(model_dict.keys())[1]
    # change model to key in variable
    df2["variable"] = [x if x != "model" else key for x in df2["variable"]]
    df = pd.concat([df1, df2])
    df = df.dropna()
    df = df.loc[:,["variable", "value", "month", "long_name"]]
    df = df.groupby(["variable", "month", "long_name"]).mean().reset_index()
    # change month to month name
    md(f"## Can the model reproduce regional {vv}?")
    (
        ggplot(df)+
        geom_line(aes(x = "month", y = "value", color = "variable"))+
        facet_wrap("long_name", scales = "free")+
        # legend at the top
        # drop the legend name
        # theme_bw()+
        theme(legend_title = element_blank())+
        theme(legend_position = "top")+
        expand_limits(y = 0)+
        # suitable breaks for months
        scale_x_continuous(breaks = [2, 4, 6, 8, 10, 12 ], labels = ["Feb", "Apr", "Jun", "Aug", "Oct", "Dec"])+ 
        # rotate the x-axis labels
          theme(axis_text_x = element_text(angle = 45))+
        labs(y = vv, x = "Month")
    )
    md(f"**Figure {i_figure}**. Spatial average {vv} for each model in each region. The values shown are monthly climatologies.")
    i_figure = i_figure + 1
  except:
    pass